In [ ]:
!pip install dash


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots  # Add this line



In [4]:

timeline_df = pd.read_csv('./datasets/timeline_df.csv')
country_df = pd.read_csv('./datasets/country_results_df.csv')

timeline_df.head()


,edition,year,country,city,countries,all_contestant,male_contestant,female_contestant,start_date,end_date
0,65,2024,United Kingdom,Bath,108,609,528,81.0,2024-07-11,2024-07-22
1,64,2023,Japan,Chiba,112,618,550,67.0,2023-07-02,2023-07-13
2,63,2022,Norway,Oslo,104,589,521,68.0,2022-07-06,2022-07-16
3,62,2021,Russian Federation,A distributed IMO administered from St Petersburg,107,619,555,64.0,2021-07-14,2021-07-24
4,61,2020,Russian Federation,A distributed IMO administered from St Petersburg,105,616,560,56.0,2020-09-19,2020-09-28


In [6]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming timeline_df and country_df are already loaded as in your previous code
timeline_df = pd.read_csv('./datasets/timeline_df.csv')
country_df = pd.read_csv('./datasets/country_results_df.csv')

# Calculate the correlation
correlation = timeline_df['countries'].corr(timeline_df['all_contestant'])

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Number of Countries and Participants Over Time"),
    dcc.Graph(id='timeline-chart'),
    html.P(f"Correlation between number of countries and participants: {correlation:.5f}") # Display the correlation
])

@app.callback(
    Output('timeline-chart', 'figure'),
    Input('timeline-chart', 'id')
)
def update_graph(input_id):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=timeline_df['year'], y=timeline_df['all_contestant'], name='Participants'), secondary_y=False)
    fig.add_trace(go.Scatter(x=timeline_df['year'], y=timeline_df['countries'], name='Number of Countries'), secondary_y=True)

    fig.update_layout(title_text="Number of Countries and Participants Over Time",
                      xaxis_title="Year",
                      yaxis_title="Participants",
                      yaxis2_title="Number of Countries")

    return fig

if __name__ == '__main__':
    app.run(debug=True,port=1001)


In [7]:
# prompt: add a map that highlights countries that participated in the competition in each year, the chart have a slide bar to change the year.

import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Countries Participating in the Competition"),
    dcc.Slider(
        id='year-slider',
        min=timeline_df['year'].min(),
        max=timeline_df['year'].max(),
        step=1,
        value=timeline_df['year'].max(), # Start with the most recent year
        marks=None,
        tooltip={
        "always_visible": True,
        "style": {"color": "LightSteelBlue", "fontSize": "20px"},

    },
    ),
    dcc.Graph(id='country-map')
])

@app.callback(
    Output('country-map', 'figure'),
    Input('year-slider', 'value')
)
def update_map(selected_year):
    # Filter the country_df for the selected year
    country_df_in_year = country_df[country_df['year'] == selected_year]

    # Create a list of ISO Alpha-3 country codes that participated
    countries_participated = country_df_in_year['country'].unique().tolist()

    # Use plotly.express to create the choropleth map
    fig = px.choropleth(
        locations=countries_participated,  # Use country codes
        locationmode='country names',  # Specify the locationmode
        color=countries_participated, # Color countries that participated
        color_discrete_sequence=["blue"], # Assign the color
        scope='world',
        title=f'Countries Participating in {selected_year}',
        height=800,  # Increased height
        width=1200,  # Increased width

    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=1002)


In [9]:
# prompt: create a line chart with 6 lines, y axis is each line represent the total points that all participants gain in each problems, x axis is year. The total points for problem 1 is calculated by sum column p1 of all country in that year, same for other problems

import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming timeline_df and country_df are already loaded
timeline_df = pd.read_csv('./datasets/timeline_df.csv')
country_df = pd.read_csv('./datasets/country_results_df.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Average points for each problem over time"),
    dcc.Graph(id='problem-points-chart')
])

@app.callback(
    Output('problem-points-chart', 'figure'),
    Input('problem-points-chart', 'id')
)
def update_graph(input_id):
    fig = go.Figure()

    for problem in ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']:  # Assuming 'p1' to 'p6' are problem columns
        # Group by year and sum the problem points for all countries in each year
        problem_points = country_df.groupby('year')[problem].sum()/timeline_df.set_index('year')['all_contestant']
        fig.add_trace(go.Scatter(x=problem_points.index, y=problem_points.values, mode='lines', name=f'Problem {problem.upper()}'))

    fig.update_layout(
        title_text="Average points for each problem over time",
        xaxis_title="Year",
        yaxis_title="Total Points"
    )

    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8051) # Use a different port


In [11]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming timeline_df and country_df are already loaded
timeline_df = pd.read_csv('./datasets/timeline_df.csv')
country_df = pd.read_csv('./datasets/country_results_df.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Country Medal Performance Over Time"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in country_df['country'].unique()],
        value=country_df['country'].unique()[0]  # Set a default value
    ),
    dcc.Graph(id='medal-chart'),
    dcc.Graph(id='medal-points-chart') # Add a new graph component
])

@app.callback(
    Output('medal-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    # Filter data for the selected country
    country_data = country_df[country_df['country'] == selected_country]

    # Create the line graph
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['awards_gold'],
                             mode='lines', name='Gold', line=dict(color='gold')))
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['awards_silver'],
                             mode='lines', name='Silver', line=dict(color='silver')))
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['awards_bronze'],
                             mode='lines', name='Bronze', line=dict(color='brown')))
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['awards_honorable_mentions'],
                             mode='lines', name='Honorable Mentions', line=dict(color='red')))

    fig.update_layout(
        title=f'Medal Performance of {selected_country} Over Time',
        xaxis_title='Year',
        yaxis_title='Number of Medals'
    )

    return fig
@app.callback(
    Output('medal-points-chart', 'figure'),
    Input('country-dropdown', 'value') # Use the same dropdown as the medal chart
)
def update_medal_points_chart(selected_country):
    # Filter data for the selected country
    country_data = country_df[country_df['country'] == selected_country]

    # Calculate medal points
    country_data['medal_points'] = (country_data['awards_gold'] * 8 +
                                   country_data['awards_silver'] * 4 +
                                   country_data['awards_bronze'] * 2 +
                                   country_data['awards_honorable_mentions'] * 1)

    # Create the medal points chart
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['medal_points'],
                             mode='lines+markers', name='Medal Points',line=dict(color='gold')))

    fig.update_layout(
        title=f'Medal Points of {selected_country} Over Time',
        xaxis_title='Year',
        yaxis_title='Medal Points'
    )
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8052) # Use a different port


C:\Users\ice\AppData\Local\Temp\ipykernel_20076\3040228723.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ice\AppData\Local\Temp\ipykernel_20076\3040228723.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ice\AppData\Local\Temp\ipykernel_20076\3040228723.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d